# GLMSingle Analysis Script

## Import all necessary libraries and setup directories

In [3]:
import sys
sys.path.append('/Users/danieljanini/GLMsingle/python')
sys.path.append('/Users/danieljanini/fracridge/fracridge')

In [4]:
import numpy as np
import re
import nibabel
import scipy
import scipy.io as sio
from scipy.ndimage import gaussian_filter
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from os.path import join, exists, split
import time
import urllib.request
import warnings
from tqdm import tqdm
from pprint import pprint
warnings.filterwarnings('ignore')
import matplotlib
%matplotlib inline
from scipy.interpolate import PchipInterpolator
from glmsingle.glmsingle import GLM_single

## Functions and directories

In [9]:
from scipy.interpolate import PchipInterpolator

def fmri_interpolate(input_mat, tr_orig=2.0, tr_new=0.5):
    """
    Interpolates 4D fMRI data (X, Y, Z, T) to double the temporal resolution.
    Assumes slice time correction was done to the middle of the TR.

    Parameters:
    - input_mat: 4D numpy array (X, Y, Z, T)
    - tr_orig: Original TR (default 1.0s)
    - tr_new: New TR (default 0.5s)

    Returns:
    - output_mat: 4D numpy array (X, Y, Z, new T)
    """
    assert input_mat.ndim == 4, "Input must be a 4D array."

    num_x, num_y, num_z, num_t = input_mat.shape

    # Create original and new time axes
    original_time = np.arange(0.5, num_t, 1.0) * tr_orig
    new_time = np.arange(0.5, original_time[-1] + tr_new, tr_new)

    # Preallocate the output matrix
    output_mat = np.zeros((num_x, num_y, num_z, len(new_time)), dtype=input_mat.dtype)

    # Interpolate for each voxel
    for x in range(num_x):
        for y in range(num_y):
            for z in range(num_z):
                time_series = input_mat[x, y, z, :]
                if np.any(time_series):  # Only interpolate if there is non-zero data
                    interpolator = PchipInterpolator(original_time, time_series)
                    output_mat[x, y, z, :] = interpolator(new_time)

    return output_mat


In [10]:
homedir = split(os.getcwd())[0]
datadir = join(homedir,'miniblock','derivatives')
presdir = join(homedir, 'Behavior', 'designmats')

## Starting the loop 

In [11]:
runtypes = [ 'er']
sm_fwhm = 2 # voxels 
tr_old = 2 # before resampling
tr_new = 0.5 # after resampling
subjects = ["02"] # subjects 9 and 16 excluded due to motion

In [12]:
for sub in subjects: 
    
    subString = f'sub-{sub}'
    for runTypeIdx in runtypes:
        for smoothing in range(1,2):
            print(f'These are {runTypeIdx} - runs and smoothing is {smoothing}')

            # Specify stimulus duration 
            if runTypeIdx == 'er':
                stimdur = 1
            else: 
                stimdur = 4

            # get files for design matrices and fmri data
            pattern = presdir + f'/P0{sub}_ConditionRich_Run*_{runTypeIdx}.csv'
            matches = glob.glob(pattern)
            matches.sort()
            
            # store data and designs
            data = []
            design = []
            for i in range(len(matches)):
                designMat = pd.read_csv(matches[i], header=None)
                print(f"Size of Design Matrix before upsampling: {designMat.shape}")
                num = re.search(r'Run_(\d+)', matches[i])
                # reminder: the runNum of the designmats is not the same as for the functional data as localizer runs were interspersed 
                runNum = int(num.group(1))
                if (runNum >3) & (runNum < 7) & (sub != '01'): 
                    runNum += 1 # localizer run after 3rd functional run
                elif (runNum >= 7):
                    runNum += 2 # localizer run after 6th functional run (7th overall)
                elif (sub == '01') & (runNum >4) & (runNum < 7):
                    runNum +=1
                
                # get the nii.gz file 
                if runNum < 10: 
                    file_path = join(datadir, subString, 'func', subString + f'_task-func_run-0{runNum}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
                else: 
                    file_path = join(datadir, subString, 'func', subString + f'_task-func_run-{runNum}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
                
                # Load the file 
                origData = nibabel.load(file_path)
                # Interpolation 
                interpData = fmri_interpolate(origData.get_fdata(), tr_old, tr_new)
                print(interpData.shape[-1])

                # Resampling the Design Matrix 
                upsample_factor = 2
                time_points, conditions = designMat.shape

                upsampled_matrix = np.zeros((time_points * upsample_factor, conditions))

                # Fill design matrix with values and remove the first row 
                upsampled_matrix[::upsample_factor, :] = designMat
                design.append(upsampled_matrix[1:-1, :])
                print(f"Size of Design Matrix after upsampling: {design[i].shape}")

                if smoothing == 1: 
                    print('doing smoothing')
                    sigma = sm_fwhm/np.sqrt(8 * np.log(2))
                    numX,numY,numz,numT = interpData.shape
                    smoothedData = np.full(interpData.shape, np.nan)
                    for tIdx in range(numT):
                        cData = interpData[:,:,:,tIdx]
                        smoothedData[:,:,:,tIdx] = gaussian_filter(cData, sigma)
                    data.append(smoothedData)

                    outputdir = join(homedir, 'miniblock', 'Outputs', subString, f'sm_{sm_fwhm}_vox_{subString}_{runTypeIdx}')
                
                else: 
                    data.append(interpData)
                    outputdir = join(homedir, 'miniblock', 'Outputs', subString, f'unsmoothed_{subString}_{runTypeIdx}')
                
                assert design[i].shape[0] == interpData.shape[-1], "Design matrix and fMRI timepoints do not match!"
                    
            print(f'There are {len(data)} runs in total\n')
            print(f'N = {data[0].shape[3]} TRs per run\n')
            print(f'The dimensions of the data for each run are: {data[0].shape}\n')
            print(f'The stimulus duration is {stimdur} seconds\n')
            print(f'XYZ dimensionality is: {data[0].shape[:3]} (one slice only in this example)\n')
            print(f'Numeric precision of data is: {type(data[0][0,0,0,0])}\n')    

            opt = dict()
            opt['wantlibrary'] = 1
            opt['wantglmdenoise'] = 1
            opt['wantfracridge'] = 1
            opt['wantfileoutputs'] = [1, 1, 1, 1]
            opt['wantmemoryoutputs'] = [1, 1, 1, 1]


            glmsingle_obj = GLM_single(opt)

            pprint(glmsingle_obj.params)

            start_time = time.time()

            os.makedirs(outputdir, exist_ok=True)
            results_glmsingle = glmsingle_obj.fit(
                design, 
                data, 
                stimdur, 
                tr_new, 
                outputdir=outputdir
            )

            elapsed_time = time.time() - start_time
            print('\telapsed_time: '
            f'{time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}'
            )               

These are er - runs and smoothing is 1
Size of Design Matrix before upsampling: (388, 40)
774
Size of Design Matrix after upsampling: (774, 40)
doing smoothing
Size of Design Matrix before upsampling: (388, 40)
774
Size of Design Matrix after upsampling: (774, 40)
doing smoothing
Size of Design Matrix before upsampling: (388, 40)
774
Size of Design Matrix after upsampling: (774, 40)
doing smoothing
There are 3 runs in total

N = 774 TRs per run

The dimensions of the data for each run are: (77, 95, 82, 774)

The stimulus duration is 1 seconds

XYZ dimensionality is: (77, 95, 82) (one slice only in this example)

Numeric precision of data is: <class 'numpy.float64'>

{'R2thresh': 0,
 'brainR2': [],
 'brainexclude': False,
 'brainthresh': [99.0, 0.1],
 'chunklen': 50000,
 'extra_regressors': False,
 'firdelay': 30,
 'firpct': 99,
 'fracs': array([1.  , 0.95, 0.9 , 0.85, 0.8 , 0.75, 0.7 , 0.65, 0.6 , 0.55, 0.5 ,
       0.45, 0.4 , 0.35, 0.3 , 0.25, 0.2 , 0.15, 0.1 , 0.05]),
 'hrffitmask':

chunks: 100%|██████████| 11/11 [15:14<00:00, 83.11s/it]



*** Saving results to /Users/danieljanini/Documents/Thesis/miniblock/Outputs/sub-02/sm_2_vox_sub-02_er/TYPEB_FITHRF.npy. ***

*** DETERMINING GLMDENOISE REGRESSORS ***

*** CROSS-VALIDATING DIFFERENT NUMBERS OF REGRESSORS ***



chunks: 100%|██████████| 11/11 [14:08<00:00, 77.12s/it]



*** FITTING TYPE-C MODEL (GLMDENOISE) ***



chunks: 100%|██████████| 11/11 [01:36<00:00,  8.75s/it]



*** Saving results to /Users/danieljanini/Documents/Thesis/miniblock/Outputs/sub-02/sm_2_vox_sub-02_er/TYPEC_FITHRF_GLMDENOISE.npy. ***

*** FITTING TYPE-D MODEL (GLMDENOISE_RR) ***



chunks: 100%|██████████| 11/11 [29:02<00:00, 158.43s/it]



*** Saving results to /Users/danieljanini/Documents/Thesis/miniblock/Outputs/sub-02/sm_2_vox_sub-02_er/TYPED_FITHRF_GLMDENOISE_RR.npy. ***

*** All model types done ***

*** return model types in results ***

	elapsed_time: 01:07:31
